In [ ]:
# 参考サイト https://github.com/cnloni/tensorflow-bezier/blob/master/main1.py

t = tf.constant(t0, tf.float32)


Matrix([
[-px0*(t - 1)**3 + 3*px1*t*(t - 1)**2 - 3*px2*t**2*(t - 1) + px3*t**3],
[-py0*(t - 1)**3 + 3*py1*t*(t - 1)**2 - 3*py2*t**2*(t - 1) + py3*t**3]])

In [4]:
import tensorflow as tf

t = tf.Variable(t0, tf.float32, name='t') # 標本点に最寄りの曲線上の点のパラメータ（標本の数）
cp = tf.Variable(ps0, tf.float32, name='cp')   # (4,2)   制御点４つと軸の上端の座標
cp = tf.Variable(ps0, tf.float32, name='cp')   # (4,2)   制御点４つと軸の上端の座標
agl1 = tf.Variable(an0, tf.float32, name='angle1')  # 軸の
agl2 = tf.Variable(an0, tf.float32, name='angle2')
s = 1 - t
T = tf.stack([s * s * s, 3 * s * s * t, 3 * s * t * t, t * t * t])
r = tf.matmul(T, bs, transpose_a=True)